In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import xmltodict
import json

## Import the first DF

In [3]:
bgg = pd.read_csv("data/BGG100.csv", encoding = 'ISO-8859-1')

# first I'm going to import the data frame about boardgames that downloaded from kaggle

bgg

,Rank,Rating,Name,Subtitle,Year,MinPlayers,MaxPlayers,BestPlayers,MinPlayTime,MaxPlayTime,MinAge,Weight,Type
0,1,8.511,Gloomhaven,Vanquish monsters with strategic cardplay. Ful...,2017,1,4,3,60,120,14,3.87,"Strategy, Thematic"
1,2,8.442,Pandemic Legacy: Season 1,Mutating diseases are spreading around the wor...,2015,2,4,4,60,60,13,2.83,"Strategy, Thematic"
2,3,8.418,Brass: Birmingham,"Build networks, grow industries, and navigate ...",2018,2,4,"3, 4",60,120,14,3.90,Strategy
3,4,8.273,Terraforming Mars,Compete with rival CEOs to make Mars habitable...,2016,1,5,3,120,120,12,3.24,Strategy
4,5,8.262,Twilight Imperium: Fourth Edition,"Build an intergalactic empire through trade, r...",2017,3,6,6,240,480,14,4.26,"Strategy, Thematic"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,7.529,The Isle of Cats,Rescue as many island cats as you can before L...,2019,1,4,3,60,90,8,2.31,"Family, Strategy"
96,97,7.528,Russian Railroads,"Acquire locomotives & industry, build track, a...",2013,2,4,4,90,120,12,3.41,Strategy
97,98,7.526,Tigris & Euphrates,Keep your Mesopotamian civilisation in perfect...,1997,2,4,4,90,90,12,3.51,Strategy
98,99,7.516,Decrypto,Decipher your opponents' code before they deci...,2018,3,8,"4, 6",15,45,12,1.80,Party


## Create functions to clean and unify the DF

In [4]:
bgg.columns

Index(['Rank', 'Rating', 'Name', 'Subtitle', 'Year', 'MinPlayers',
       'MaxPlayers', 'BestPlayers', 'MinPlayTime', 'MaxPlayTime', 'MinAge',
       'Weight', 'Type'],
      dtype='object')

### Clean Dataframe

In [5]:
def clean_dataframe(df, columns_to_erase=[], rename=True):
    # change the name of the columns if its necessary, into lower case and striping the whitespaces
    if rename:
        df.columns = [i.lower().replace(" ", "_").strip() for i in df.columns]
    # drop rows with fewer than 2 non-NaN values
    df.dropna(thresh=2, inplace=True)
    # drop specified columns
    df.drop(columns=columns_to_erase, inplace=True, errors="ignore")

    return df

columns_to_drop = ["subtitle", "weight"]  
bgg = clean_dataframe(bgg, columns_to_drop, rename=True)

# Here I'm reusing the basic changes I want to the column names, as I did in the previous project
# to make columns easier to manage and more unified

bgg

,rank,rating,name,year,minplayers,maxplayers,bestplayers,minplaytime,maxplaytime,minage,type
0,1,8.511,Gloomhaven,2017,1,4,3,60,120,14,"Strategy, Thematic"
1,2,8.442,Pandemic Legacy: Season 1,2015,2,4,4,60,60,13,"Strategy, Thematic"
2,3,8.418,Brass: Birmingham,2018,2,4,"3, 4",60,120,14,Strategy
3,4,8.273,Terraforming Mars,2016,1,5,3,120,120,12,Strategy
4,5,8.262,Twilight Imperium: Fourth Edition,2017,3,6,6,240,480,14,"Strategy, Thematic"
...,...,...,...,...,...,...,...,...,...,...,...
95,96,7.529,The Isle of Cats,2019,1,4,3,60,90,8,"Family, Strategy"
96,97,7.528,Russian Railroads,2013,2,4,4,90,120,12,Strategy
97,98,7.526,Tigris & Euphrates,1997,2,4,4,90,90,12,Strategy
98,99,7.516,Decrypto,2018,3,8,"4, 6",15,45,12,Party


In [6]:
bgg["type"].value_counts()

type
Strategy                   36
Strategy                   23
Thematic                   12
Strategy,  Thematic        11
Family,  Strategy           6
Family                      3
Strategy,  Wargames         2
Customizable                2
Abstract,  Family           2
Thematic,  Wargames         1
Customizable,  Thematic     1
Party                       1
Name: count, dtype: int64

### Unify values

In [21]:
def unify_values(df):
    df.loc[df['type'].str.contains('Strategy', case=False), 'type'] = 'Strategy'
    # with this function I'm going to unify all the results under just one name
    # all values that contains Strategy in their name, will change to just "Strategy"
    # and the same to the other 2 values.
    df.loc[df['type'].str.contains('Thematic', case=False), 'type'] = 'Thematic'
    df.loc[df['type'].str.contains('Family', case=False), 'type'] = 'Family'
    df['rating'] = df['rating'].round(1)
    # here I round all values in rating to 1 number
    df['bestplayers'] = df['bestplayers'].str.split(',').str[0]
    df['bestplayers'] = df['bestplayers'].replace(['4','5', '6'], '4+')
    
unify_values(bgg)

In [22]:
bgg

,rank,rating,name,year,minplayers,maxplayers,bestplayers,minplaytime,maxplaytime,minage,type
0,1,8.5,Gloomhaven,2017,1,4,3,60,120,14,Strategy
1,2,8.4,Pandemic Legacy: Season 1,2015,2,4,4+,60,60,13,Strategy
2,3,8.4,Brass: Birmingham,2018,2,4,3,60,120,14,Strategy
3,4,8.3,Terraforming Mars,2016,1,5,3,120,120,12,Strategy
4,5,8.3,Twilight Imperium: Fourth Edition,2017,3,6,4+,240,480,14,Strategy
...,...,...,...,...,...,...,...,...,...,...,...
95,96,7.5,The Isle of Cats,2019,1,4,3,60,90,8,Strategy
96,97,7.5,Russian Railroads,2013,2,4,4+,90,120,12,Strategy
97,98,7.5,Tigris & Euphrates,1997,2,4,4+,90,90,12,Strategy
98,99,7.5,Decrypto,2018,3,8,4+,15,45,12,Party


## Import the second DF from the API

In [11]:
top_ten = pd.read_csv("data/top_ten_votes.csv")

top_ten

,Unnamed: 0,game_name,num_players,value,num_votes
0,0,Gloomhaven,1,Best,143
1,1,Gloomhaven,1,Recommended,643
2,2,Gloomhaven,1,Not Recommended,356
3,3,Gloomhaven,2,Best,373
4,4,Gloomhaven,2,Recommended,822
...,...,...,...,...,...
169,169,War of the Ring: Second Edition,4,Recommended,125
170,170,War of the Ring: Second Edition,4,Not Recommended,198
171,171,War of the Ring: Second Edition,4+,Best,1
172,172,War of the Ring: Second Edition,4+,Recommended,3


In [ ]:
top_ten = top_ten.drop(columns=['Unnamed: 0'])



In [28]:
top_ten["game_name"].unique()

array(['Gloomhaven', 'Pandemic Legacy: Season 1', 'Brass: Birmingham',
       'Terraforming Mars', 'Twilight Imperium: Fourth Edition',
       'Gloomhaven: Jaws of the Lion', 'Gaia Project',
       'Star Wars: Rebellion',
       'Through the Ages: A New Story of Civilization',
       'War of the Ring: Second Edition'], dtype=object)

## Reducing names for the plots

In [32]:
# As I'm experimenting in the visualization process, the names
# are too long and make difficult to see them clearly in the plots
# so I'm going to make a function that will reduce them

def reducing_names (df):
    df['game_name'] = df.apply(lambda row: row['game_name'].split(':')[0] if row['game_name'] != 'Brass: Birmingham' else row['game_name'], axis=1)
    # in this case I'm applying a lambda that will split the name after the colon and taking the first part
    # if it's distinct to the value "Brass: Birmingham", if not, it will mantain the same name
    df['num_players'] = df['num_players'].replace(['4','5', '6'], '4+')
    # this modification replace the results "4, 5, 6" and convert them into "4+"

    
reducing_names(top_ten)

In [33]:

top_ten


,game_name,num_players,value,num_votes
0,Gloomhaven,1,Best,143
1,Gloomhaven,1,Recommended,643
2,Gloomhaven,1,Not Recommended,356
3,Gloomhaven,2,Best,373
4,Gloomhaven,2,Recommended,822
...,...,...,...,...
169,War of the Ring,4+,Recommended,125
170,War of the Ring,4+,Not Recommended,198
171,War of the Ring,4+,Best,1
172,War of the Ring,4+,Recommended,3


## Exporting the DF modified

In [34]:
bgg.to_csv('data/bgg_modified.csv', index=False)

top_ten.to_csv('data/top_ten_modified.csv', index=False)
